In [1]:
import pandas as pd
import os
from datetime import datetime, timedelta
import re


In [2]:
# def load_options_data(ticker, start_date, end_date, data_dir):
#     all_data = []
#     current_date = datetime.strptime(start_date, "%Y-%m-%d")
#     end = datetime.strptime(end_date, "%Y-%m-%d")
    
#     print(f"Fetching data for {ticker} from {start_date} to {end_date}...")
    
#     while current_date <= end:
#         date_str = current_date.strftime("%Y-%m-%d")
#         year = current_date.strftime("%Y")
#         month = current_date.strftime("%m") 
        
#         file_path = os.path.join(data_dir, year, month, f"{date_str}.csv.gz")
        
#         if os.path.exists(file_path):
#             print(f"Processing {file_path}...")
#             try:
#                 # Read the CSV file
#                 # The file has columns: ticker,conditions,correction,exchange,price,sip_timestamp,size
#                 df = pd.read_csv(file_path, compression='gzip')
                
#                 # Filter for the specific ticker
#                 # The ticker column format is like 'O:A251121C00125000'
#                 # We look for tickers starting with 'O:NVDA'
#                 target_prefix = f"O:{ticker}"
#                 df_filtered = df[df['ticker'].str.startswith(target_prefix)]
                
#                 if not df_filtered.empty:
#                     all_data.append(df_filtered)
#             except Exception as e:
#                 print(f"Error reading {file_path}: {e}")
#         else:
#             # print(f"File not found: {file_path}") # Optional: reduce noise
#             pass
            
#         current_date += timedelta(days=1)
    
#     if all_data:
#         return pd.concat(all_data, ignore_index=True)
#         # return combined_df
# #        combined_df = pd.concat(all_data, ignore_index=True)
#         # Remove duplicates that might exist in the raw data
#         # initial_len = len(combined_df)
#         # combined_df = combined_df.drop_duplicates()
#         # if len(combined_df) < initial_len:
#         #     print(f"Removed {initial_len - len(combined_df)} duplicate rows from raw data.")
#         # return combined_df
#     else:
#         # print("No data found for the specified criteria.")
#         return pd.DataFrame()



In [3]:
def load_options_data(ticker, start_date, end_date, data_dir):
    all_data = []
    current_date = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    
    # Handle single ticker or list of tickers
    if isinstance(ticker, str):
        tickers = [ticker]
    else:
        tickers = ticker

    print(f"Fetching data for {tickers} from {start_date} to {end_date}...")
    
    # Create regex pattern for filtering
    # Matches strings starting with O:TICKER for any ticker in the list
    pattern = '|'.join([f"^O:{t}" for t in tickers])
    
    while current_date <= end:
        date_str = current_date.strftime("%Y-%m-%d")
        year = current_date.strftime("%Y")
        month = current_date.strftime("%m") 
        
        file_path = os.path.join(data_dir, year, month, f"{date_str}.csv.gz")
        
        if os.path.exists(file_path):
            print(f"Processing {file_path}...")
            try:
                # Read the CSV file
                # The file has columns: ticker,conditions,correction,exchange,price,sip_timestamp,size
                df = pd.read_csv(file_path, compression='gzip')
                
                # Filter for the specific ticker(s)
                # The ticker column format is like 'O:A251121C00125000'
                # We look for tickers starting with 'O:NVDA'
                df_filtered = df[df['ticker'].str.contains(pattern, regex=True)]
                
                if not df_filtered.empty:
                    all_data.append(df_filtered)
            except Exception as e:
                print(f"Error reading {file_path}: {e}")
        else:
            # print(f"File not found: {file_path}") # Optional: reduce noise
            pass
            
        current_date += timedelta(days=1)
    
    if all_data:
        return pd.concat(all_data, ignore_index=True)
        # return combined_df
#        combined_df = pd.concat(all_data, ignore_index=True)
        # Remove duplicates that might exist in the raw data
        # initial_len = len(combined_df)
        # combined_df = combined_df.drop_duplicates()
        # if len(combined_df) < initial_len:
        #     print(f"Removed {initial_len - len(combined_df)} duplicate rows from raw data.")
        # return combined_df
    else:
        # print("No data found for the specified criteria.")
        return pd.DataFrame()

In [4]:
TICKER = ["SPXW", "SPX", "NVDA", "NDX", "TSLA", "AAPL", "AMZN", "AMD", "GOOGL"]
START_DATE = "2025-11-19"
END_DATE = "2025-11-23"
DATA_DIR = "/blue/yixuanli/wyanbin/ResearchData/Algo/us_options_opra/day_aggs_v1"
# Load and merge data
stock_options = load_options_data(TICKER, START_DATE, END_DATE, DATA_DIR)


Fetching data for ['SPXW', 'SPX', 'NVDA', 'NDX', 'TSLA', 'AAPL', 'AMZN', 'AMD', 'GOOGL'] from 2025-11-19 to 2025-11-23...
Processing /blue/yixuanli/wyanbin/ResearchData/Algo/us_options_opra/day_aggs_v1/2025/11/2025-11-19.csv.gz...
Processing /blue/yixuanli/wyanbin/ResearchData/Algo/us_options_opra/day_aggs_v1/2025/11/2025-11-20.csv.gz...
Processing /blue/yixuanli/wyanbin/ResearchData/Algo/us_options_opra/day_aggs_v1/2025/11/2025-11-20.csv.gz...
Processing /blue/yixuanli/wyanbin/ResearchData/Algo/us_options_opra/day_aggs_v1/2025/11/2025-11-21.csv.gz...
Processing /blue/yixuanli/wyanbin/ResearchData/Algo/us_options_opra/day_aggs_v1/2025/11/2025-11-21.csv.gz...


In [5]:
# Configuration
TICKER = "SPXW"
# TICKER= "NVDA"
START_DATE = "2025-11-21"
END_DATE = "2025-11-23"
DATA_DIR = "/blue/yixuanli/wyanbin/ResearchData/Algo/us_options_opra/trades_v1"
DATA_DIR = "/blue/yixuanli/wyanbin/ResearchData/Algo/us_options_opra/day_aggs_v1"
# Load and merge data
stock_options = load_options_data(TICKER, START_DATE, END_DATE, DATA_DIR)

# Display results
print(f"Total rows loaded: {len(stock_options)}")
if not stock_options.empty:
    print(stock_options.head())

Fetching data for ['SPXW'] from 2025-11-21 to 2025-11-23...
Processing /blue/yixuanli/wyanbin/ResearchData/Algo/us_options_opra/day_aggs_v1/2025/11/2025-11-21.csv.gz...
Total rows loaded: 8460
                  ticker  volume     open    close     high      low  \
0  O:SPXW251121C01400000       2  5167.35  5233.35  5233.35  5167.35   
1  O:SPXW251121C03700000       1  2857.94  2857.94  2857.94  2857.94   
2  O:SPXW251121C04000000       1  2557.37  2557.37  2557.37  2557.37   
3  O:SPXW251121C04200000      46  2414.64  2417.70  2432.84  2414.64   
4  O:SPXW251121C04250000      50  2364.37  2366.22  2382.07  2364.37   

          window_start  transactions  
0  1763701200000000000             2  
1  1763701200000000000             1  
2  1763701200000000000             1  
3  1763701200000000000            10  
4  1763701200000000000            11  
Total rows loaded: 8460
                  ticker  volume     open    close     high      low  \
0  O:SPXW251121C01400000       2  5167.35  5

In [6]:
stock_options[stock_options['volume']>100]

,ticker,volume,open,close,high,low,window_start,transactions
64,O:SPXW251121C06125000,101,431.40,435.29,437.01,431.40,1763701200000000000,8
102,O:SPXW251121C06400000,381,165.27,223.00,257.95,128.00,1763701200000000000,109
108,O:SPXW251121C06430000,340,143.77,193.23,218.85,110.40,1763701200000000000,30
110,O:SPXW251121C06440000,945,121.55,166.42,207.15,108.41,1763701200000000000,83
112,O:SPXW251121C06450000,440,129.59,174.58,200.90,95.10,1763701200000000000,86
...,...,...,...,...,...,...,...,...
8321,O:SPXW260630C07300000,202,119.40,132.10,132.40,119.40,1763701200000000000,11
8332,O:SPXW260630C07800000,195,33.54,31.18,33.54,30.95,1763701200000000000,10
8335,O:SPXW260630C08200000,201,9.81,12.25,12.25,9.81,1763701200000000000,9
8417,O:SPXW260930P06200000,270,291.34,290.47,294.94,288.96,1763701200000000000,18


In [7]:
# show the exactly duplicated rows 
exact_duplicates = stock_options[stock_options.duplicated(keep=False)]
print(exact_duplicates)

Empty DataFrame
Columns: [ticker, volume, open, close, high, low, window_start, transactions]
Index: []


In [8]:
# count how many duplicates in ticker 
stock_options['ticker'].value_counts()

ticker
O:SPXW251121C01400000    1
O:SPXW251121C03700000    1
O:SPXW251121C04000000    1
O:SPXW251121C04200000    1
O:SPXW251121C04250000    1
                        ..
O:SPXW260930P07070000    1
O:SPXW260930P07325000    1
O:SPXW260930P07330000    1
O:SPXW260930P07375000    1
O:SPXW260930P08000000    1
Name: count, Length: 8460, dtype: int64

In [5]:
# drop the duplicates based on all columns
stock_options = stock_options.drop_duplicates()
print(f"Total rows after dropping duplicates: {len(stock_options)}")

Total rows after dropping duplicates: 83161


In [ ]:

# Convert sip_timestamp to datetime
# Assuming the timestamp is in nanoseconds and converting to US/Eastern
stock_options['sip_timestamp'] = pd.to_datetime(stock_options['sip_timestamp'], unit='ns').dt.tz_localize('UTC').dt.tz_convert('US/Eastern')


In [ ]:
import numpy as np
from concurrent.futures import ProcessPoolExecutor
import os
import sys

# Ensure the current directory is in the path so we can import the utility module
# This helps if the notebook is running from a different working directory
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

# Import the processing function from the external file
# We try both relative and absolute imports to be safe
try:
    from parallel_utils import process_chunk
except ImportError:
    try:
        from analysis_backtesting.parallel_utils import process_chunk
    except ImportError:
        # If both fail, we might need to add the directory of the notebook explicitly
        notebook_dir = os.path.dirname(os.path.abspath("__file__"))
        if notebook_dir not in sys.path:
            sys.path.append(notebook_dir)
        from parallel_utils import process_chunk

# Parallel execution
# Use ProcessPoolExecutor to utilize multiple CPU cores
n_workers = os.cpu_count()
print(f"Parallelizing with {n_workers} workers...")

# Split the dataframe into chunks
chunks = np.array_split(stock_options, n_workers)

with ProcessPoolExecutor(max_workers=n_workers) as executor:
    results = list(executor.map(process_chunk, chunks))

# Combine results
stock_options = pd.concat(results)

print(stock_options[['sip_timestamp', 'ticker', 'ticker_symbol', 'expiration', 'option_type', 'strike']].head())

In [ ]:
pattern = r'O:?([A-Z]+)(\d{6})([CP])(\d{8})'
extracted = stock_options['ticker'].str.extract(pattern)
stock_options['ticker_symbol'] = extracted[0]
stock_options['expiration'] = pd.to_datetime(extracted[1], format='%y%m%d')
stock_options['option_type'] = extracted[2].map({'C': 'call', 'P': 'put'})
stock_options['strike'] = pd.to_numeric(extracted[3]) / 1000.0

print(stock_options[['sip_timestamp', 'ticker', 'ticker_symbol', 'expiration', 'option_type', 'strike']].head())

In [ ]:
stock_options

In [ ]:
stock_options

In [ ]:
def load_stock_data(ticker, start_date, end_date, data_dir):
    all_data = []
    current_date = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    
    print(f"Fetching stock data for {ticker} from {start_date} to {end_date}...")
    
    while current_date <= end:
        date_str = current_date.strftime("%Y-%m-%d")
        year = current_date.strftime("%Y")
        month = current_date.strftime("%m")
        
        file_path = os.path.join(data_dir, year, month, f"{date_str}.csv.gz")
        
        if os.path.exists(file_path):
            print(f"Processing {file_path}...")
            try:
                # Read the CSV file
                # Columns: ticker,volume,open,close,high,low,window_start,transactions
                df = pd.read_csv(file_path, compression='gzip')
                
                # Filter for the specific ticker
                df_filtered = df[df['ticker'] == ticker].copy()
                
                if not df_filtered.empty:
                    all_data.append(df_filtered)
            except Exception as e:
                print(f"Error reading {file_path}: {e}")
        
        current_date += timedelta(days=1)
        
    if all_data:
        return pd.concat(all_data, ignore_index=True)
    else:
        print("No stock data found for the specified criteria.")
        return pd.DataFrame()

# Configuration for Stock Data
STOCK_DATA_DIR = "/blue/yixuanli/wyanbin/ResearchData/Algo/us_stocks_sip/day_aggs_v1"


In [ ]:
START_DATE="2025-11-15"
# Load stock data (using same date range as options)
stock_data = load_stock_data(TICKER, START_DATE, END_DATE, STOCK_DATA_DIR)

# Convert window_start to datetime (assuming nanoseconds like options)
if not stock_data.empty:
    stock_data['window_start'] = pd.to_datetime(stock_data['window_start'], unit='ns').dt.tz_localize('UTC').dt.tz_convert('US/Eastern')

print(f"Total stock rows loaded: {len(stock_data)}")
if not stock_data.empty:
    print(stock_data.head())

In [ ]:
#rename stock_data ticker column to ticker_symbol for merging
stock_data = stock_data.rename(columns={'ticker': 'ticker_symbol'})

In [ ]:
stock_data.head()

In [ ]:
stock_options.head()

In [ ]:
# Create normalized date columns for accurate daily matching
stock_options['trade_date'] = stock_options['sip_timestamp'].dt.normalize()
stock_data['stock_date'] = stock_data['window_start'].dt.normalize()

# Merge to get strictly previous trading day's data
# allow_exact_matches=False ensures that if trade_date == stock_date, it's skipped, 
# finding the most recent stock_date strictly less than trade_date.
merged_data = pd.merge_asof(
    stock_options.sort_values('trade_date'),
    stock_data.sort_values('stock_date'),
    left_on='trade_date',
    right_on='stock_date',
    by='ticker_symbol',
    direction='backward',
    allow_exact_matches=False
)

In [ ]:
merged_data

In [ ]:
# generate maturity days for each option trade, ignore the timezone difference

#merged_data['trade_date'] = merged_data['trade_date'].dt.tz_localize(None)
#merged_data['expiration'] = merged_data['expiration'].dt.tz_localize(None)
merged_data['maturity_days'] = (merged_data['expiration'] - merged_data['trade_date'].dt.tz_localize(None)).dt.days

In [ ]:
# Calculate dollar volume (price * size * 100)
merged_data['dollar_volume'] = merged_data['size'] * merged_data['price'] /10000 # in millions

# generate summary stat for the trades and volumes for each maturity days at aggregation 
maturity_summary = merged_data.groupby('maturity_days').agg(
    total_trades=pd.NamedAgg(column='ticker', aggfunc='count'),
    total_volume=pd.NamedAgg(column='size', aggfunc='sum'),
    total_dollar_volume=pd.NamedAgg(column='dollar_volume', aggfunc='sum'),
    average_price=pd.NamedAgg(column='price', aggfunc='mean')
).reset_index()

In [ ]:
maturity_summary